# Overview

We invoke Huggingface ML ecosystem libs in kimchima. We want to make sure that the kimchima can be a buffer zone for ML dependencies. So, we use poetry to manage all the ML libs in kimchima.

## Note

First, we upgrade to Python 3.11. After the installing process was finished. We restart the notebook.

In [ ]:
%%capture
!mamba create -n py311 -y
!source /opt/conda/bin/activate py311 && mamba install python=3.11 jupyter mamba -y

!sudo rm /opt/conda/bin/python3
!sudo ln -sf /opt/conda/envs/py311/bin/python3 /opt/conda/bin/python3
!sudo rm /opt/conda/bin/python3.10
!sudo ln -sf /opt/conda/envs/py311/bin/python3 /opt/conda/bin/python3.10
!sudo rm /opt/conda/bin/python
!sudo ln -sf /opt/conda/envs/py311/bin/python3 /opt/conda/bin/python

In [ ]:
!python --version

In [ ]:
import sys
sys.version

In [ ]:
!pip install -U -q kimchima==0.4.1

# Computing Embeddings

In [1]:
from kimchima import (
    ModelFactory, 
    TokenizerFactory,
    StreamerFactory,
    EmbeddingsFactory,
    PipelinesFactory,
    Devices
)


pretrained_model_name_or_path = "sentence-transformers/all-MiniLM-L6-v2"

model = ModelFactory.auto_model(pretrained_model_name_or_path=pretrained_model_name_or_path)
tokenizer= TokenizerFactory.auto_tokenizer(pretrained_model_name_or_path=pretrained_model_name_or_path)

# computing embeddings for single text
embeddings = EmbeddingsFactory.get_text_embeddings(
    model=model,
    tokenizer=tokenizer,
    prompt='Melbourne',
    device='cuda'
)
print(embeddings.shape)

# computing embeddings for multiple texts
embeddings = EmbeddingsFactory.get_text_embeddings(
    model=model,
    tokenizer=tokenizer,
    prompt=['Melbourne', 'Sydney'],
    device='cuda'
)
print(embeddings.shape)

# Checking the device: GPU, mps and CPU
device = Devices.get_device()
print(device)


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Some weights of the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 were not used when initializing BertModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

torch.Size([1, 384])
torch.Size([2, 384])
Devices.GPU


In [2]:
# get capability of GPU(Nvidia)
capability = Devices.get_capability()
print(capability)

(6, 0)


# Inference by using Streaming

In [3]:
# streamer
model= ModelFactory.auto_model_for_causal_lm(pretrained_model_name_or_path="gpt2")
tokenizer= TokenizerFactory.auto_tokenizer(pretrained_model_name_or_path="gpt2")
streamer= StreamerFactory.text_streamer(tokenizer=tokenizer, skip_prompt=False, skip_prompt_tokens=False)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
pipe=PipelinesFactory.text_generation(
    model=model, 
    tokenizer=tokenizer, 
    text_streamer=streamer
    )

pip.device

In [ ]:
pipe("Melbourne is the capital of Victoria")

In [ ]:
conversation_model="facebook/blenderbot-400M-distill"
msg = "why Melbourne is a good place to travel?"
prompt = "Melbourne is often considered one of the most livable cities globally, offering a high quality of life."
res = PipelinesFactory.chat_response(
    conversation_model=conversation_model,
    messages=msg,
    prompt=prompt
    )

In [ ]:
res

# Acknowledge
* https://www.kaggle.com/discussions/general/402718